In [1]:
import numpy as np, pandas as pd, re, pickle, spacy, string
from lingua import Language, LanguageDetectorBuilder
from sklearn import neighbors
from scipy import spatial
from bs4 import BeautifulSoup
import nltk
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
nlp = spacy.load("en_core_web_lg")

### Working classifier but need faster

In [21]:
from transformers import pipeline

classifier = pipeline('zero-shot-classification', model='facebook/bart-large-mnli')

In [76]:
labels = {
"application fee": 0,
"course fee": 1,
"deposit fee": 2,
"donation": 3,
"membership": 4,
"product fee": 5,
"registration fee": 6,
"service fee": 7,
"setup fee": 8,
"contract fee": 9,
"subscription": 10}
ll = list(labels.keys())
c2l = {v:i for i,v in labels.items()}

In [22]:
labels = ll
hypothesis_template = 'This text is about {}.'

In [49]:
np.random.randint(1000, len(k))

9548

In [50]:
sequence = ' '.join([token.text for token in nlp(' '.join(k[9548]))])
sequence

"Submit Full Name E - mail Address Phone Number Donation Founded in 1970 , Senior Citizens Home Assistance Service , Inc. ( SCHAS ) mission is to improve the quality of life for the elderly , persons with disabilities , and their families by providing accessible care and supportive services through compassionate personnel . SCHAS ’ programs include in - home caregiving services , independent living at Cedar Point for low - income seniors in Tazewell , Tennessee , transportation , and the nationally - award winning SCHAS Out Shopping free grocery shopping - delivery service for any East Tennessee resident that is age 60 or older . Financial Aid for In - Home Caregiving Services : SCHAS helps the elderly and the adult disabled stay safely in their homes and provide respite for their families with kind , trained , in - home caregivers . \xa0  SCHAS serves 20 counties throughout the East Tennessee region . \xa0  Unlike other care giving agencies , SCHAS offers financial aid to clients who 

In [51]:
prediction = classifier(sequence, ll, sequenceothesis_template=hypothesis_template)

print(pd.DataFrame(prediction['labels'], prediction['scores']))

                         0
0.291330          donation
0.117854   application fee
0.110512         setup fee
0.107790       product fee
0.098279        course fee
0.068068       deposit fee
0.061622       service fee
0.041803      contract fee
0.039011      subscription
0.033177        membership
0.030555  registration fee


In [52]:
test = np.mean([i.vector for i in nlp(' '.join(k[9548]))], axis=0)

In [53]:
c2l[cm.predict(test.reshape(1,-1))[0]]

'donation'

# Get json from api

In [36]:
import nest_asyncio 
nest_asyncio.apply() #needed as this is jupyter

import asyncio
import aiohttp

res = []

In [125]:
ids = pd.read_csv("./payment_forms_dataset.csv", header=None)

In [126]:
urls = []
for i, row in ids.iterrows():
    urls.append(f"https://api.jotform.com/form/{row[0]}/questions?apiKey=06c38963c5de8d06c64b64c8fa9f31a8")
    
#/formsuda çek 

In [ ]:
async def get_json(session, url):
    async with session.get(url) as resp:
        json = await resp.json()
        return json

async def main1():
    async with aiohttp.ClientSession() as session:
        tasks = []
        for url in urls[:50000]:
            tasks.append(asyncio.ensure_future(get_json(session, url)))
        result = await asyncio.gather(*tasks)
        res.append(result)

async def main2():
    async with aiohttp.ClientSession() as session:
        tasks = []
        for url in urls[50000:100000]:
            tasks.append(asyncio.ensure_future(get_json(session, url)))
        result = await asyncio.gather(*tasks)
        res.append(result)

async def main3():
    async with aiohttp.ClientSession() as session:
        tasks = []
        for url in urls[100000:]:
            tasks.append(asyncio.ensure_future(get_json(session, url)))
        result = await asyncio.gather(*tasks)
        res.append(result)

asyncio.run(main1())
asyncio.run(main2())
asyncio.run(main3())

In [ ]:
res = [*res[0], *res[1], *res[2]]

In [54]:
res = [i for i in res if i['responseCode'] == 200]

In [18]:
with open('data1.pkl', 'wb') as handle:
    pickle.dump(res, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Checkpoint 1

## Get text from json

In [2]:
with open('data1.pkl', 'rb') as handle:
    k = pickle.load(handle)

In [3]:
k = [i['content'] for i in k]

In [4]:
with open('langs2.pkl', 'rb') as handle:
    langs = pickle.load(handle)

In [5]:
pp = []
for i in k:
    if type(i) == dict:
        p = []
        for n in i.values():
            for j in n:
                if j == 'text' or j == 'subHeader':
                    p.append(n[j])
        pp.append(p)

In [6]:
del k

In [7]:
pp = [v for i,v in enumerate(pp) if langs[i] == Language.ENGLISH]

In [8]:
def clean_html(html):

    # parse html content
    soup = BeautifulSoup(html, "html.parser")

    for data in soup(['style', 'script', 'code', 'a']):
        # Remove tags
        data.decompose()

    # return data by retrieving the tag content
    return ' '.join(soup.stripped_strings)

In [9]:
pp = [[clean_html(j) for j in i] for i in pp]

C:\Python\envs\myenv\lib\site-packages\bs4\__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(
C:\Python\envs\myenv\lib\site-packages\bs4\__init__.py:404: MarkupResemblesLocatorWarning: The input looks more like a URL than markup. You may want to use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  warnings.warn(


In [19]:
words = ['', 'page break', 'email', 'submit', 'street adress', 'city', 'zip', 'state', 'section separator form style remove', 'name']
pp = [[j for j in i if j not in words] for i in pp]

In [25]:
pp[0]

['Submit Donation',
 'Your Full Name',
 'Phone Number',
 'E-mail',
 'Donation Amount',
 'Once all eligible nominees’ wishes have been fulfilled, Herald Charities Inc. may use funds for other charitable purposes.\xa0Herald Charities Inc., is a separately incorporated 501 (c)(3) charitable organization. All donations to Herald Charities, Inc. are tax deductible as allowed by law. No goods or services were provided in return for this contribution. Herald Charities, Inc. has complied with the registration requirements of Chapter 496, Florida Statures, the Solicitations of Contributions Act. A COPY OF THE OFFICIAL REGISTRATION AND FINANCIAL INFORMATION MAY BE OBTAINED FROM THE DIVISION OF CONSUMER SERVICES BY CALLING TOLL-FREE 1-800-435-7352 WITHIN THE STATE. REGISTRATION DOES NOT IMPLY ENDORSEMENT, APPROVAL, OR RECOMMENDATION BY THE STATE.',
 'mh-enh-bottom-logo',
 'Anonymous',
 'For 40 years, the Miami Herald Wish Book Program has spotlighted the needs of less fortunate individuals in Sou

In [26]:
with open('data2.pkl', 'wb') as handle:
    pickle.dump(pp, handle, protocol=pickle.HIGHEST_PROTOCOL)

## Get hand data

In [56]:
with open('hand_train', 'rb') as handle:
    hand = pickle.load(handle)

In [57]:
hand = pd.DataFrame(hand)

In [58]:
hand.columns = ['url', 'label']

In [59]:
hand['id'] = hand.url.apply(lambda x: x.split(u'/')[-1])

In [60]:
hand['url'] = hand.id.apply(lambda x: f"https://api.jotform.com/form/{x}/questions?apiKey=06c38963c5de8d06c64b64c8fa9f31a8")

In [61]:
jsons = []

In [62]:
async def get_json(session, url):
    async with session.get(url) as resp:
        json = await resp.json()
        return json

async def main1():
    async with aiohttp.ClientSession() as session:
        tasks = []
        for url in hand.url:
            tasks.append(asyncio.ensure_future(get_json(session, url)))
        result = await asyncio.gather(*tasks)
        jsons.append(result)
        
asyncio.run(main1())

In [63]:
hand['json'] = jsons[0]

In [64]:
hand.json = hand.json.apply(lambda x: x['content'])

In [65]:
pp = []
for i in hand.json:
    if type(i) == dict:
        p = []
        for n in i.values():
            for j in n:
                if j == 'text' or j == 'subHeader':
                    p.append(n[j])
        pp.append(p)
    else:
        pp.append([])

In [66]:
handj = [[clean_html(j) for j in i] for i in pp]

C:\Python\envs\myenv\lib\site-packages\bs4\__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


In [67]:
words = ['', 'page break', 'email', 'submit', 'street adress', 'city', 'zip', 'state', 'section separator form style remove', 'name']
handj = [[j for j in i if j not in words] for i in handj]

In [68]:
hand['text'] = handj

In [69]:
hand.text = hand.text.apply(lambda x: '. '.join(x))

In [71]:
detector = LanguageDetectorBuilder.from_all_languages().build()
hand['langs'] = hand.text.apply(lambda x: detector.detect_language_of(x))

In [72]:
hand = hand[hand.langs == Language.ENGLISH]

In [73]:
hand.drop(columns=['url', 'id', 'json', 'langs'], axis=1, inplace=True)

C:\temp\ipykernel_24036\2793397263.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hand.drop(columns=['url', 'id', 'json', 'langs'], axis=1, inplace=True)


In [74]:
hand.head()

,label,text
0,[service fee],18x24 Yard Signs - EXPIRES Dec 31st @ 11:59pm....
1,[service fee],Deprecated Drive-Thru Booking. Test Date. Page...
2,[service fee],BuySocialTraffic Service Order Form. E-mail. S...
3,[donation],Donation Amount
4,[donation],Submit. Enter Donation Amount Here. What would...


In [75]:
with open('hand_train.pkl', 'wb') as handle:
    pickle.dump(hand, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Checkpoint 2

In [13]:
with open('data2.pkl', 'rb') as handle:
    k = pickle.load(handle)

In [15]:
def del_list_indexes(l, id_to_del):
    somelist = [i for j, i in enumerate(l) if j not in id_to_del]
    return somelist

In [16]:
k = del_list_indexes(k, range(1,500)) # I dont care about ~200 observations :/

In [17]:
k = pd.Series(k)

In [18]:
k.head()

0    [Submit Donation, Your Full Name, Phone Number...
1    [DHC - Pay Your Bill, Submit, Patient Name, Da...
2    [Please enter the amount due as discussed with...
3    [Order Details, Full Name, E-mail, Attachments...
4    [Donate to Brooklyn Youth Chorus, Name, Email,...
dtype: object

In [85]:
spaced_k = pd.Series([' '.join(i) for i in k])

20000 Obs will be enough i think

In [89]:
tokens = [nlp(i) for i in spaced_k[:20000]]

In [93]:
hand.head()

,label,text
0,[service fee],18x24 Yard Signs - EXPIRES Dec 31st @ 11:59pm....
1,[service fee],Deprecated Drive-Thru Booking. Test Date. Page...
2,[service fee],BuySocialTraffic Service Order Form. E-mail. S...
3,[donation],Donation Amount
4,[donation],Submit. Enter Donation Amount Here. What would...


In [94]:
labels = {
'donation':0,
'subscription':1,
'membership':2,
'service fee':3,
'application fee':4,
'registration fee':5,
'product fee':6,
'contract fee':7,
'setup fee':8,
'course fee':9,
'deposit fee':10}
ll = list(labels.keys())
c2l = {v:i for i,v in labels.items()}

In [97]:
hand = hand.join(hand['label'].str.join('|').str.get_dummies())

In [99]:
hand.head()

,label,text,application fee,course fee,deposit fee,donation,membership,product fee,registration fee,service fee,setup fee,subsciption,upgrading fee
0,[service fee],18x24 Yard Signs - EXPIRES Dec 31st @ 11:59pm....,0,0,0,0,0,0,0,1,0,0,0
1,[service fee],Deprecated Drive-Thru Booking. Test Date. Page...,0,0,0,0,0,0,0,1,0,0,0
2,[service fee],BuySocialTraffic Service Order Form. E-mail. S...,0,0,0,0,0,0,0,1,0,0,0
3,[donation],Donation Amount,0,0,0,1,0,0,0,0,0,0,0
4,[donation],Submit. Enter Donation Amount Here. What would...,0,0,0,1,0,0,0,0,0,0,0


In [100]:
hand['text'] = hand.text.apply(lambda x: nlp(x))

In [101]:
hand.head()

,label,text,application fee,course fee,deposit fee,donation,membership,product fee,registration fee,service fee,setup fee,subsciption,upgrading fee
0,[service fee],"(18x24, Yard, Signs, -, EXPIRES, Dec, 31st, @,...",0,0,0,0,0,0,0,1,0,0,0
1,[service fee],"(Deprecated, Drive, -, Thru, Booking, ., Test,...",0,0,0,0,0,0,0,1,0,0,0
2,[service fee],"(BuySocialTraffic, Service, Order, Form, ., E,...",0,0,0,0,0,0,0,1,0,0,0
3,[donation],"(Donation, Amount)",0,0,0,1,0,0,0,0,0,0,0
4,[donation],"(Submit, ., Enter, Donation, Amount, Here, ., ...",0,0,0,1,0,0,0,0,0,0,0


In [103]:
with open('hand_train.pkl', 'wb') as handle:
    pickle.dump(hand, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [120]:
with open('data3.pkl', 'wb') as handle:
    pickle.dump(tokens, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Checkpoint 3

In [3]:
with open('hand_train.pkl', 'rb') as handle:
    hand = pickle.load(handle)

In [5]:
with open('data3.pkl', 'rb') as handle:
    k = pickle.load(handle)

EOFError: Ran out of input

In [10]:
len(k)

20000

In [13]:
hand['vector'] = hand.text.apply(lambda x: np.mean([i.vector for i in x], axis=0))

In [22]:
k = pd.Series(k)
k['vector'] = k.apply(lambda x: np.mean([i.vector for i in x], axis=0))

C:\Python\envs\myenv\lib\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,


In [25]:
with open('hand_train.pkl', 'wb') as handle:
    pickle.dump(hand, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [27]:
with open('data4.pkl', 'wb') as handle:
    pickle.dump(k, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Machine learning

In [97]:
with open('hand_train.pkl', 'rb') as handle:
    hand = pickle.load(handle)

In [2]:
with open('good_train.pkl', 'rb') as handle:
    good_train = pickle.load(handle)

In [3]:
good_train['vector'] = good_train.Text.apply(lambda x: np.mean([i.vector for i in nlp(x)], axis=0))

In [11]:
with open('data4.pkl', 'rb') as handle:
    k = pickle.load(handle)

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

In [5]:
from imblearn.over_sampling import SMOTE

In [6]:
good_train.Label.str.get_dummies().sum(axis=0)

application fee       58
contract fee          17
course fee           125
deposit fee           89
donation             681
membership           878
product fee          395
registration fee     107
service fee          486
setup fee             52
subscription        1112
dtype: int64

In [7]:
good_train.Label.value_counts()

subscription        1112
membership           878
donation             681
service fee          486
product fee          395
course fee           125
registration fee     107
deposit fee           89
application fee       58
setup fee             52
contract fee          17
Name: Label, dtype: int64

In [6]:
labels = {
"application fee": 0,
"course fee": 1,
"deposit fee": 2,
"donation": 3,
"membership": 4,
"product fee": 5,
"registration fee": 6,
"service fee": 7,
"setup fee": 8,
"contract fee": 9,
"subscription": 10}
ll = list(labels.keys())
c2l = {v:i for i,v in labels.items()}

In [7]:
X_resampled, y_resampled = SMOTE().fit_resample(list(good_train.loc[:,'vector']), good_train.Label.map(labels))

In [8]:
x_train, x_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=.3, random_state=405)

In [9]:
lsvm = RandomForestClassifier(n_estimators=500)

cm = lsvm
cm.fit(list(x_train), y_train)
preds = cm.predict(list(x_test))
print(classification_report(y_test, preds, target_names=ll))

                  precision    recall  f1-score   support

 application fee       0.97      1.00      0.99       339
      course fee       0.94      1.00      0.97       315
     deposit fee       0.97      1.00      0.98       335
        donation       0.87      0.79      0.83       329
      membership       0.74      0.77      0.75       355
     product fee       0.79      0.89      0.84       338
registration fee       0.98      0.99      0.99       315
     service fee       0.82      0.84      0.83       327
       setup fee       0.99      1.00      1.00       341
    contract fee       1.00      1.00      1.00       336
    subscription       0.67      0.51      0.58       340

        accuracy                           0.89      3670
       macro avg       0.89      0.89      0.89      3670
    weighted avg       0.88      0.89      0.88      3670



In [23]:
k.shape

(129462,)

In [19]:
test = np.mean([i.vector for i in nlp(' '.join(k[45680]))], axis=0)

In [20]:
c2l[cm.predict(test.reshape(1,-1))[0]]

'membership'

In [131]:
test = np.mean([i.vector for i in nlp(' '.join(k[45679]))], axis=0)

In [132]:
c2l[cm.predict(test.reshape(1,-1))[0]]

'product fee'